In [2]:
import os

import pandas as pd

from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import wandb

os.environ["WANDB_SILENT"] = "true"

wandb.login(key=os.getenv(key="WANDB_KEY"))

True

In [43]:
data = pd.read_csv("../data/clean.csv")
data.head()

,price,bed,bath,acre_lot,zip_code,house_size,location
0,105000.0,3.0,2.0,0.12,601,920.0,2351
1,80000.0,4.0,2.0,0.08,601,1527.0,2351
2,67000.0,2.0,1.0,0.15,795,748.0,2392
3,145000.0,4.0,2.0,0.10,731,1800.0,2412
4,179000.0,4.0,3.0,0.46,612,2520.0,2423


In [44]:
X = data.drop("price",1)
y = data["price"]

/tmp/ipykernel_1493/2212425159.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = data.drop("price",1)


In [45]:
wandb.config = {
    'learning_rate': 0.07632400095462799,
    'random_seed': 0,
    'depth': 6,
    'subsample': 0.800000011920929,
    'use_best_model': False,
    'silent': True,
    'eval_metric': "RMSE"
}

model = CatBoostRegressor(**wandb.config)

In [46]:
from pickle import dump


fold = KFold(n_splits=15,random_state=99,shuffle=True)

for index,(train_idx,test_idx) in enumerate(fold.split(X,y)):
    wandb.init(project="capstone-mlops", entity="heisguyy",name=f"Cat_Exp_{index}",tags=["hyperparameter tuning","cross validation"])
    X_train, X_val, y_train, y_val = X.iloc[train_idx,:], X.iloc[test_idx,:], y[train_idx], y[test_idx]
    model.fit(X_train,y_train)
    y_preds = model.predict(X_val)
    error = mean_squared_error(y_val,y_preds,squared=False)
    wandb.summary["error"] = error
    wandb.finish()

0
1
2
3
4


Thread SenderThread:
Traceback (most recent call last):
  File "/home/ubuntu/.local/share/virtualenvs/capstone-vm-x0ttIRgE/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 51, in run
    self._run()
  File "/home/ubuntu/.local/share/virtualenvs/capstone-vm-x0ttIRgE/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 102, in _run
    self._process(record)
  File "/home/ubuntu/.local/share/virtualenvs/capstone-vm-x0ttIRgE/lib/python3.10/site-packages/wandb/sdk/internal/internal.py", line 310, in _process
    self._sm.send(record)
  File "/home/ubuntu/.local/share/virtualenvs/capstone-vm-x0ttIRgE/lib/python3.10/site-packages/wandb/sdk/internal/sender.py", line 304, in send
    send_handler(record)
  File "/home/ubuntu/.local/share/virtualenvs/capstone-vm-x0ttIRgE/lib/python3.10/site-packages/wandb/sdk/internal/sender.py", line 318, in send_request
    send_handler(record)
  File "/home/ubuntu/.local/share/virtualenvs/capstone-vm-x0ttIRgE/lib/pyth